In [1]:
#Checking wheather GPU is working or not
!nvidia-smi


Fri Dec 19 17:21:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# installing dataset and transformer
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.

In [3]:
# to remove version conflict of Protobuf so, downgrade version of Protobuf
!pip install protobuf==3.20.3 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [4]:
!pip uninstall -y pyarrow datasets
!pip cache purge

# install fresh, modern, compatible versions
!pip install --no-cache-dir "datasets>=2.21.0" "pyarrow>=15.0.0"
!pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: pyarrow 22.0.0
Uninstalling pyarrow-22.0.0:
  Successfully uninstalled pyarrow-22.0.0
Found existing installation: datasets 4.4.1
Uninstalling datasets-4.4.1:
  Successfully uninstalled datasets-4.4.1
Files removed: 24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 12.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 341.7 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich

In [5]:
# Importing all required modules
import os
import sys
import transformers
import torch  # pytorch Import
import sacrebleu
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset # for loading the dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # For getting Embedding
from transformers import DataCollatorForSeq2Seq #getting sequential model and collator for loading batchwise of data
from torch.optim import AdamW # Optimizer



2025-12-19 17:21:45.700114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766164906.082797      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766164906.197294      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Enter Access Token and rerun
from huggingface_hub import login
login(new_session=False)

# Note:
* I was using the free version of Kaggle, and the memory limit was getting exhausted while training the 1B-parameter model. Because of this constraint, I switched to using the 200M-parameter model instead.

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "ai4bharat/indictrans2-indic-en-1B"
LORA_REPO  = "Vir123-dev/indictrans2_hing_en_finetune_1B"

# Load tokenizer ONLY from base model
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# Load base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# Attach LoRA adapters
model = PeftModel.from_pretrained(
    base_model,
    LORA_REPO
)

model = model.to("cuda")
model.eval()


tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

...

modeling_indictrans.py:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

ValueError: Can't find 'adapter_config.json' at 'Vir123-dev/indictrans2_hing_en_finetune_1B'

# The Dataset¶

* Source: https://huggingface.co/datasets/atrisaxena/mini-iitb-english-hindi

In [ ]:
raw_dataset = load_dataset("atrisaxena/mini-iitb-english-hindi")

In [ ]:
raw_dataset

# Observation for Statistics related to dataset

In [ ]:
# Import required libraries
import numpy as np
import math
import nltk
nltk.download("punkt")  # one-time

def add_stats(example):
    text = example["translation"]["hi"]
    # guard
    if text is None: text = ""
    text = text.strip() # Removes unwanted spacing
    words = text.split()
    # sentence count (approx)
    sents = nltk.tokenize.sent_tokenize(text) if text else []
    example["num_words"] = len(words)
    example["num_chars"] = len(text)
    example["num_sentences"] = len(sents)
    return example

raw_dataset = raw_dataset.map(add_stats, batched=False)

In [ ]:
# Obtaining the Statistics:

def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))

In [ ]:
from datasets import DatasetDict

In [ ]:
# Train has min length of sentences as 0 ('min': 0) so, we Remove these row from dataset
def not_empty(example):
    text = example["translation"]["hi"]
    return text is not None and len(text.strip()) > 0
    
clean_train = raw_dataset["train"].filter(not_empty)
clean_val   = raw_dataset["validation"].filter(not_empty)
clean_test  = raw_dataset["test"].filter(not_empty)

raw_dataset = DatasetDict({
    "train": clean_train,
    "validation": clean_val,
    "test": clean_test
})

In [ ]:
type(raw_dataset)

In [ ]:
# compute p99 threshold('p99': 68 so, removing other outliers(longer than 68 words))
word_lengths = np.array(raw_dataset["train"]["num_words"])
p99_threshold = int(np.percentile(word_lengths, 99))
print("Removing sentences longer than:", p99_threshold, "words")
raw_dataset["train"] = raw_dataset["train"].filter(
    lambda ex: ex["num_words"] <= p99_threshold
)


In [ ]:
# Obtaining the desired Statistics
def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))


In [ ]:
# Sample Example
raw_dataset['train'][0]

In [ ]:
from datasets import DatasetDict

# New desired sizes
N_TRAIN = 2000
N_VAL   = 150
N_TEST  = 250

# Downsample using .select()
small_train = raw_dataset["train"].select(range(N_TRAIN))
small_val   = raw_dataset["validation"].select(range(N_VAL))
small_test  = raw_dataset["test"].select(range(N_TEST))

# Create a new DatasetDict
small_dataset = DatasetDict({
    "train": small_train,
    "validation": small_val,
    "test": small_test
})

small_dataset

In [ ]:
raw_dataset = small_dataset  # As said in Task 2(for 2000 pair of sentence)

# Applying Tokenization:- How we obtain Embedding
* Attention Mask = Padding Mask x look Ahead Mask
* Input_ids = input_ids are tokenized text converted into numeric indices from tokenizer vocabulary.
* The model converts input_ids to embeddings internally through an embedding layer.

# Pipeline
* Text → Tokens → IDs → Embeddings → Transformer
* "I love India"
*      ↓              (tokenization)
* ["I","love","India"]
*      ↓              (vocab lookup)
* [34, 91, 2563]  ← input_ids
*      ↓
* [embedding vectors] ← actual embeddings used by model

In [ ]:
# sample Example;
text = "Hello Myself Virendra. A final year student at NIT Surat."
tokenizer("eng_Latn hin_Deva " + text)

In [ ]:
# Tokenize the Target language(hindi - hin_Deva) - Example
with tokenizer.as_target_tokenizer():
    print(tokenizer("मधुमक्खियों को पालने का कार्य। "))

In [ ]:

# simple inference (model already on GPU by device_map)
SRC_TAG = "hin_Deva"
TGT_TAG = "eng_Latn"
text = "My name is Virendra, I am Currently a AI/ML Researcher"

tagged = f"{SRC_TAG} {TGT_TAG} {text}"

inputs = tokenizer(tagged, return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        use_cache=False   
    )

print(tokenizer.decode(out[0], skip_special_tokens=True))


# Evaluation metrics

# Finding  BLEU AND CHRF:

1. BLEU: BLEU checks how many n-grams from the candidate sentence also appear in the reference sentence.
2. CHRF: Instead of words, CHRF compares character n-grams.

In [ ]:
model.device

In [ ]:
n_samples = 10

# determine model device safely
try:
    model_device = next(model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Model device:", model_device)

preds = []
refs = []

for i in range(min(n_samples, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # handle multiple possible row formats
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        # trans might be a dict or a string; handle both
        if isinstance(trans, dict):
            eng = trans.get("en") or trans.get("eng") or ""
            hi_ref = trans.get("hi") or trans.get("hin") or ""
        else:
            # sometimes translation is a string (rare) — treat as source
            eng = str(trans)
            hi_ref = ""
    elif isinstance(row, dict):
        # maybe keys are directly 'en' and 'hi'
        eng = row.get("en") or row.get("eng") or row.get("source") or ""
        hi_ref = row.get("hi") or row.get("hin") or row.get("target") or ""
    else:
        # fallback: row itself might be the translation dict-like
        try:
            eng = row["en"]
            hi_ref = row["hi"]
        except Exception:
            # last resort: stringify
            eng = str(row)
            hi_ref = ""

    eng = (eng or "").strip()
    hi_ref = (hi_ref or "").strip()
    refs.append(eng if eng else "")  # keep alignment

    # add required tags
    tagged = f"{SRC_TAG} {TGT_TAG} {eng}"

    # tokenize -> torch tensors -> move to model device
    tokenized = tokenizer(tagged, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = model.generate(**tokenized, max_length=128, num_beams=4, use_cache=False ,early_stopping=True)

    pred_text = tokenizer.decode(out_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
    preds.append(pred_text)


In [ ]:
#BLEU:
bleu = sacrebleu.corpus_bleu(preds, [refs])
#CHRF:
chrf = sacrebleu.corpus_chrf(preds, [refs])

print(f"\nEvaluated {len(preds)} samples")
print("BLEU:", bleu.score)
print("CHRF:", chrf.score)

# show a few examples
for i in range(min(5, len(preds))):
    print(f"\n=== SAMPLE {i+1} ===")
    print("SRC :", (raw_dataset["test"][i].get("translation", raw_dataset["test"][i]).get("hi")
                    if isinstance(raw_dataset["test"][i], dict) and "translation" in raw_dataset["test"][i]
                    else (raw_dataset["test"][i].get("hi") if isinstance(raw_dataset["test"][i], dict) else str(raw_dataset["test"][i]))))
    print("PRED:", preds[i])
    print("REF :", refs[i])


# Finding BERTScore:

* BERTScore: Uses BERT (or RoBERTa, or mBERT) embeddings to compare every token in candidate with every token in reference.

In [ ]:
!pip install bert-score
from bert_score import score

P, R, F1 = score(preds, refs, lang="hi")
print("BERTScore F1:", F1.mean().item())

# Finding BLEURT:
* BLEURT computes a similarity score using a fine-tuned BERT model that predicts human judgment of translation quality.

In [ ]:
# Required Packages for Bleurt
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

# Calculating Bleurt
import evaluate
bleurt = evaluate.load("bleurt")
results = bleurt.compute(predictions=preds, references=refs)
print(results)

# Finding COMET
* COMET predicts a score that strongly correlates with human judgment.

In [ ]:
# Required package for Comet
!pip install -q unbabel-comet
from comet import download_model, load_from_checkpoint


# choose model variable 
translation_model = globals().get("translation_model", None) or globals().get("model", None)
if translation_model is None:
    raise ValueError("No translation model found. Load your model into `model` or `translation_model` first.")

# device: try to get model device (handles DeviceMap too)
try:
    model_device = next(translation_model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SRC_TAG = "eng_Latn"
TGT_TAG = "hin_Deva"

srcs = []
preds = []
refs = []

n = 10
for i in range(min(n, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # robust extraction of source & reference
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        if isinstance(trans, dict):
            ref = (trans.get("en") or trans.get("eng") or "").strip()
            src= (trans.get("hi") or trans.get("hin") or "").strip()
        else:
            ref= str(trans).strip()
            src= ""
    elif isinstance(row, dict):
        ref = (row.get("en") or row.get("eng") or row.get("source") or "").strip()
        src = (row.get("hi") or row.get("hin") or row.get("target") or "").strip()
    else:
        # fallback
        ref = str(row).strip()
        src = ""

    srcs.append(src)
    refs.append(ref)

    # add language tags required by IndicTrans2
    tagged = f"{SRC_TAG} {TGT_TAG} {src}"

    # tokenize -> PyTorch tensors -> move to model device
    tokenized = tokenizer(tagged,
                          return_tensors="pt",
                          truncation=True,
                          padding=True,
                          max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = translation_model.generate(**tokenized, max_length=128, num_beams=4,use_cache=False , early_stopping=True)

    pred = tokenizer.decode(out_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
    preds.append(pred)

# ---- COMET evaluation ----
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

# prepare data list for COMET
data = [{"src": s, "mt": p, "ref": r} for s, p, r in zip(srcs, preds, refs)]
# note: comet_model.predict returns a dict; 'scores' contains numeric values
comet_out = comet_model.predict(data, batch_size=8)
comet_scores = comet_out["scores"] if isinstance(comet_out, dict) and "scores" in comet_out else comet_out

print("Samples evaluated:", len(preds))
print("Mean COMET score:", float(sum(comet_scores) / len(comet_scores)))

# quick side-by-side preview
for i in range(len(preds)):
    print(f"\n--- SAMPLE {i+1} ---")
    print("SRC :", srcs[i])
    print("PRED:", preds[i])
    print("REF :", refs[i])


In [ ]:
# Saving the Model and tokenizer
model.save_pretrained("pt_model")
tokenizer.save_pretrained("tokenizer")